<a href="https://colab.research.google.com/github/kwhc441/mask_police/blob/main/20220111_color_tracking_by_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#物体検出（輪郭）
import numpy as np
import cv2
from matplotlib import pyplot as plt
 
# 画像から輪郭を検出する関数
def contours(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                  # グレースケール化
    ret, img_binary = cv2.threshold(img_gray,                         # 二値化
                                    60, 255,                          # 二値化のための閾値60(調整要)
                                    cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(img_binary,                # 輪郭検出
                                           cv2.RETR_EXTERNAL,         # 外側の輪郭のみ抽出
                                           cv2.CHAIN_APPROX_SIMPLE)
    contours = np.array(contours)                                     # 輪郭情報をndarrayに変換
    x = np.mean(contours[0].T[0, 0])                                  # 輪郭のx方向平均値を算出
    y = np.mean(contours[0].T[1, 0])                                  # 輪郭のy方向平均値を算出
    return x, y
 
movie = cv2.VideoCapture('video.mp4')                                 # 動画ファイルの読み込み
 
# 動画ファイル保存用の設定
fps = int(movie.get(cv2.CAP_PROP_FPS))                                # 動画のFPSを取得
w = int(movie.get(cv2.CAP_PROP_FRAME_WIDTH))                          # 動画の横幅を取得
h = int(movie.get(cv2.CAP_PROP_FRAME_HEIGHT))                         # 動画の縦幅を取得
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')                   # 動画保存時のfourcc設定（mp4用）
video = cv2.VideoWriter('video_out.mp4', fourcc, fps, (w, h), True)   # 動画の仕様（ファイル名、fourcc, FPS, サイズ, カラー）
 
# ファイルからフレームを1枚ずつ取得して動画処理後に保存する
x_list = []
y_list = []
while True:
    ret, frame = movie.read()                                         # フレームを取得
 
    # フレームが取得できない場合はループを抜ける
    if not ret:
        break
 
    x, y = contours(frame)                                            # 輪郭検出から物体中心を算出
 
    frame = cv2.circle(frame, (int(x), int(y)), 30, (0, 255, 0), 3)   # 検出した位置にサークル描画
 
    video.write(frame)  # 動画を保存する
    x_list.append(x)
    y_list.append(y)
 
# 動画オブジェクト解放
movie.release()

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install bottle
!pip install bottle_websocket
!pip install gevent

from IPython.display import display, Javascript
from google.colab.output import eval_js

def use_cam(url, quality=0.8):
  print("start camera")
  js = Javascript('''
    async function useCam(url, quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      display_size = 500 
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size * video.videoHeight / video.videoWidth;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      //exit button
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Exit';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      //log
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc} `));
      }
      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      //for websocket connection.
      var connection = 0
      var flag_count = 0
      var send_flg = false

      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            flag_count += 1

            //wait until websocket launch
            if (flag_count == 200){
                connection = new WebSocket(url); 
                jsLog("Connect_start")
            }
            if (flag_count == 300){
                connection.onmessage = function(e) {
                    var image = new Image()
                    image.src = e.data;
                    image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                    send_flg=false
                };
                jsLog("Set_recieve")
            }
            if(flag_count > 400){
                //resize to reduce file size
                src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);
                const img = src_canvas.toDataURL('image/jpeg', quality);
                if (send_flg==false){
                    connection.send(img);
                    send_flg = true
                }
            }
          if (exit_flg){
              requestAnimationFrame(_canvasUpdate);   
          }else{
              stream.getVideoTracks()[0].stop();
              connection.close();
          }
      };
    }
    ''')
  display(js)
  data = eval_js('useCam("{}", {})'.format(url, quality))

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json;url = json.load(sys.stdin)['tunnels'][0]['public_url'];print(url);f = open('url.txt', 'w');f.write(url);f.close()"



--2022-01-12 05:24:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  40.1MB/s    in 0.3s    

2022-01-12 05:24:32 (40.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://962b-34-67-119-98.ngrok.io


In [ ]:
yimport numpy as np
import cv2

# Need adjustment
lower_light_pink = np.array([168, 50, 50])
upper_light_pink = np.array([188, 255, 255])

_LOWER_COLOR = lower_light_pink
_UPPER_COLOR = upper_light_pink

def tracking():
    cap = cv2.VideoCapture(0)

    filter = ParticleFilter()
    filter.initialize()

    while True:
        ret, frame = cap.read()

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Threshold the HSV image to get only a color
        mask = cv2.inRange(hsv, _LOWER_COLOR, _UPPER_COLOR)

        # Start Tracking
        y, x = filter.filtering(mask)
        frame = cv2.circle(frame, (int(x), int(y)), 10, (255, 0, 0), -1)

        # origin is upper left
        frame_size = frame.shape
        print ("position_x_rate")
        print (x/frame_size[1])
        print ("position_y_rate")
        print (y/frame_size[0])

        for i in range(filter.SAMPLEMAX):
            frame = cv2.circle(frame, (int(filter.X[i]), int(filter.Y[i])), 2, (0, 0, 255), -1)
        cv2.imshow("frame", frame)

        if cv2.waitKey(20) & 0xFF == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

class ParticleFilter:
    def __init__(self):
        self.SAMPLEMAX = 1000
        # frame.shape
        self.height, self.width = 480, 640

    def initialize(self):
        self.Y = np.random.random(self.SAMPLEMAX) * self.height
        self.X = np.random.random(self.SAMPLEMAX) * self.width

    # Need adjustment for tracking object velocity
    def modeling(self):
        self.Y += np.random.random(self.SAMPLEMAX) * 200 - 100 # 2:1
        self.X += np.random.random(self.SAMPLEMAX) * 200 - 100

    def normalize(self, weight):
        return weight / np.sum(weight)

    def resampling(self, weight):
        index = np.arange(self.SAMPLEMAX)
        sample = []

        # choice by weight 
        for i in range(self.SAMPLEMAX):
            idx = np.random.choice(index, p=weight)
            sample.append(idx)
        return sample

    def calcLikelihood(self, image):
        # white space tracking 
        mean, std = 250.0, 10.0
        intensity = []

        for i in range(self.SAMPLEMAX):
            y, x = self.Y[i], self.X[i]
            if y >= 0 and y < self.height and x >= 0 and x < self.width:
                intensity.append(image[int(y),int(x)])
            else:
                intensity.append(-1)

        # normal distribution
        weights = 1.0 / np.sqrt(2 * np.pi * std) * np.exp(-(np.array(intensity) - mean)**2 /(2 * std**2))
        weights[intensity == -1] = 0
        weights = self.normalize(weights)
        return weights

    def filtering(self, image):
        self.modeling()
        weights = self.calcLikelihood(image)
        index = self.resampling(weights)
        self.Y = self.Y[index]
        self.X = self.X[index]
        
        # return COG
        return np.sum(self.Y) / float(len(self.Y)), np.sum(self.X) / float(len(self.X))

In [ ]:
import numpy as np
import cv2
import json
import bottle
import gevent
from bottle.ext.websocket import GeventWebSocketServer
from bottle.ext.websocket import websocket
from multiprocessing import Pool
from PIL import Image
from io import BytesIO
import base64
lower_color = np.array([170,170,90])
upper_color = np.array([190,255,255])
_LOWER_COLOR = lower_color
_UPPER_COLOR = upper_color

socket = bottle.Bottle()
@socket.route('/', apply=[websocket])
def wsbin(ws):
    while True:
        try:
            #decode to image
            img_str = ws.receive()
            decimg = base64.b64decode(img_str.split(',')[1], validate=True)
            decimg = Image.open(BytesIO(decimg))
            decimg = np.array(decimg, dtype=np.uint8); 
            decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

            #############your process###############

            cap = cv2.VideoCapture(0)

            filter = ParticleFilter()
            filter.initialize()

            while True:
                ret, frame = cap.read()

                hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

                # Threshold the HSV image to get only a color
                mask = cv2.inRange(hsv, _LOWER_COLOR, _UPPER_COLOR)

                # Start Tracking
                y, x = filter.filtering(mask)
                frame = cv2.circle(frame, (int(x), int(y)), 10, (255, 0, 0), -1)

                # origin is upper left
                frame_size = frame.shape
                print ("position_x_rate")
                print (x/frame_size[1])
                print ("position_y_rate")
                print (y/frame_size[0])

                for i in range(filter.SAMPLEMAX):
                    frame = cv2.circle(frame, (int(filter.X[i]), int(filter.Y[i])), 2, (0, 0, 255), -1)
                cv2.imshow("frame", frame)

                if cv2.waitKey(20) & 0xFF == 27:
                    break
            cap.release()
            cv2.destroyAllWindows()


            #############your process###############

            #encode to string
            _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
            img_str = encimg.tostring()
            img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
            ws.send(img_str)
        except:
            pass
            #print("error")

if __name__ == '__main__':
    # get ngrok url
    f = open("url.txt", "r")
    url = f.read()
    f.close()
    url = "wss" + url[5:]
    # prepare multiprocess
    _pool = Pool(processes=2)
    _pool.apply_async(use_cam, (url, 0.8))
    socket.run(host='0.0.0.0', port=6006, server=GeventWebSocketServer)

start camera


<IPython.core.display.Javascript object>

Bottle v0.12.19 server starting up (using GeventWebSocketServer())...
Listening on http://0.0.0.0:6006/
Hit Ctrl-C to quit.


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/gevent/threadpool.py", line 157, in _before_run_task
    _sys.settrace(_get_thread_trace())


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/gevent/threadpool.py", line 162, in _a